ECE143, Spring 2018

Prof. Unpingco

Group 14

# Scraping Crime Data

### Overview

- SANDAG and the justice department publish a lot of their data in .pdf format.
- There are .csv format tables available, but it is difficult to tell the browser program what data to include/exclude.
- We need to convert the tables in the PDFs into pandas tables or numpy arrays before we can combine the crime data with the influencing variables (e.g. weather, homelessness, income, etc.).
- Relevant functions are being produced in the `cleantables.py` module.

### Step 1: Convert PDF to CSV

- If we can get the files as a simple text, then we can use regular expressions and the like to more easily parse out the data.
- We want to do this step in python rather than by hand since there are potentially many files form which we can extract the data. Doing so in a batch format would be ideal and the most replicable.
- The first python module I found is [`tabula`-py](https://github.com/chezou/tabula-py). It has a function `convert_into_by_batch()` which converts every file in a directory into CSV format.
    - Tabula-py is a wrapper for a java program. To get this to work, java must already be installed.
    - Install from command line with `pip install tabula-py`

In [3]:
#Example: Converting my class schedule into a CSV
#Note: Kernel's current directory is 'ECE-143-Group-14', which contains the 'TEST' direectory
import tabula as tab

tab.convert_into_by_batch( "TEST", output='csv') #CSV is default, just being explicit
#The one PDF file in TEST is "SP18 sched.pdf"
#Also contians a normal .txt file, which gets ignored

with open('TEST\\SP18 sched.csv', 'r') as sched: #Check the resulting CSV
    for i in range(5):
        print next(sched) #see first five lines

"",Monday,Tuesday,Wednesday,Thursday,,Friday,Saturday,Sunday

"",,11:00 - 12:20 Enrolled,,1 1:00 - 12:20 Enrolled,,,,

11am,,CHEM  151,,CHEM  151,,,,

"",,LE / NSB 2303,,LE / NSB 2303,,,,

"",,"Weizma n, Haim",,"Weizma n, Haim",,,,



- Manually saving each table from \[insert file name(s)\] as a separate PDF into the sub-directory \[insert dir name\].
- For each PDF, a file with extension '.csv' will be created. New CSV files that share names with old ones will replace the older versions.
- Saved some PDF tables published by the [CJSC](https://oag.ca.gov/cjsc/pubs) covering California statewide data going back to 1952 and San Diego region-specific data going back to 2013. Saved under the `"crime_data"` directory.

Note: "ARJISPublicCrime041818.txt" is a CSV downloaded directly from the [SANDAG website](http://www.sandag.org/index.asp?classid=14&subclassid=21&projectid=446&fuseaction=projects.detail). It is ignored by the tabula module.

In [4]:
#Converting PDF data tables to CSV
#Current directory is "ECE-143-Group-14"
import tabula as tab
import os

tab.convert_into_by_batch( "crime_data", output='csv')
datafiles = os.listdir("crime_data") #list out file names
for data in datafiles:
    print data #View all of the data files saved

ARJISPublicCrime041818.txt
CAcrimeIndex52-96_cjsc.csv
CAcrimeIndex52-96_cjsc.pdf
CAcrimes66-15_cjsc.csv
CAcrimes66-15_cjsc.pdf
SDjurisdiction_2013_cjbulletin.csv
SDjurisdiction_2013_cjbulletin.pdf
SDjurisdiction_2014_cjbulletin.csv
SDjurisdiction_2014_cjbulletin.pdf
SDjurisdiction_2015_cjbulletin.csv
SDjurisdiction_2015_cjbulletin.pdf
SDjurisdiction_2016_cjbulletin.csv
SDjurisdiction_2016_cjbulletin.pdf
SDjurisdiction_2017_cjbulletin.csv
SDjurisdiction_2017_cjbulletin.pdf


- Checking the output from `tabula.convert_into_by_batch()` in Excel to clearly see how tabula organized the rows and columns.
    - Statewide data:
![alt text](https://github.com/YijingLi0726/ECE-143-Group-14/blob/scraping-crime-data/Crime%20Scraping%20Process/CAexcel_screencap.png?raw=true, 'Statewide Excel Screencap')
    - County regional data:
![alt text](https://github.com/YijingLi0726/ECE-143-Group-14/blob/scraping-crime-data/Crime%20Scraping%20Process/SDexcel_screencap.png?raw=true,'SD Excel Screencap')
    - General observations
        - Captions below and titles above the tables are removed.
        - Multi-line column headers are split into different rows.
        - Did not split crime rates from the total counts, which were in separate columns within the same cell in the original PDF.
        - In the state data, there are a few empty spacer columns.

- The UCSD daily police logs are provided as one PDF per day, going back through the past months. I would like to have a program go through and download each one for me instead of doing so myself.
    - The URLs are pretty straightforward as far as format:
```
http://www.police.ucsd.edu/docs/reports/CallsandArrests/CallsForService/[month]%20[day],%20[year].pdf
```
    - For example, the report for May 5, 2018 would be:
```
http://www.police.ucsd.edu/docs/reports/CallsandArrests/CallsForService/May%205,%202018.pdf
```



----

## Step 2: Converting CSV Files into Pandas Data Tables

- To most easily collapse and split the data from the CSV, we want to manipulate these files as pandas data tables.
- The main function used to do this for the crime data is `cleantables.batch_clean_tab_csv()`.

#### `cleantables.batch_clean_tab_csv()`
**Description:** Processes CSV files into pandas dataframes. Requires many inputs, but provides fairly customizable control over what and how files get processed.

**Parameters**

**csvs:** *list of str*
- File paths by which the function may access CSV files.
- Order is especially relevant if year specified outside of the table (see `by_year`).

**heads:** *list of str, default=[ ]*
- List of desired column headers for the dataframe output.
- Must be exactly the same length as the number of columns in the FINAL dataframe, as applied after the dataframe is contructed.
- Function requires that each column have a unique label.

**hrow:** *int, >= 0, default=3*
- The row immediately one index prior, or above one row above, the data values.
- Default is set to 3 due to this being the correct header row for CSVs derived from [CJSC documents](https://oag.ca.gov/cjsc/pubs).
- If the headers found in this row are unsatisfactory (e.g. they contain excape characters, headers are missing, multi-line headers) see `heads`.

**by_year:** *list of int, default=range(2013,2018)*
- Specifies year that each CSV covers. Matches according to index in list, so order is important.
- Default values come from the years for which CJSC publilshed annual crime summaries subdivided into each jurisdiction.
- The length of this list must be the same as the length of the CSV file list input in `csvs`.

 
**Output:** *pandas DataFrame*
- Cleaned and organized data from the CSV output of tabula-py.
- If `by_year` was a non-empty list, the output would have two indicies, one for year and one for row number. Otherwise, only the row index is used.

In [3]:
#Used the os module to go to directory 'ECE-143-group-14'
import pandas as pd
from cleantables import batch_clean_tab_csv
sd_jurisdiction_crime = batch_clean_tab_csv(
    ['crime_data\\SDjurisdiction_2013_cjbulletin.csv','crime_data\\SDjurisdiction_2014_cjbulletin.csv',
     'crime_data\\SDjurisdiction_2015_cjbulletin.csv','crime_data\\SDjurisdiction_2016_cjbulletin.csv',
     'crime_data\\SDjurisdiction_2017_cjbulletin.csv'],
    #Name the CSVs that we need to combine into one dataframe; working with San Diego County data
    heads=['jurisdiction','homicide','rape','robbery','aggravated_assault','residential_burglary',
           'non-residential_burglary','total_burglary','larceny>$400','larceny>=$400',
           'total_larceny','motor_vehicle_theft','arson','FBI_crime_index','CA_crime_index'],
    #Name all of the column headings that are in the tables
    hrow=3,
    #Designate the row where the headings will be inserted
    splitcol=[],
    #State data was not separated correctly, but the SD tables did not have this issue
    by_year=range(2013,2018)
    #The years were removed with the table titles, so we put this is as well
    )
print sd_jurisdiction_crime

                                    jurisdiction  homicide     rape robbery  \
year row                                                                      
2013 0                                  Carlsbad       1.0       23      44   
     1                               Chula Vista       2.0       29     248   
     2                                  Coronado       0.0        2       4   
     3                                  El Cajon       2.0       29     143   
     4                                 Escondido       6.0       39     201   
     5                                   La Mesa       1.0        7      64   
     6                             National City       1.0       11     137   
     7                                 Oceanside       9.0       41     182   
     8                                 San Diego      39.0     3 16   1,456   
     9                           Sheriff - Total       9.0     1 63     554   
     10                                  Del Mar    

In [5]:
#Pickling the data for team members' use
import cPickle as pickle
pickle.dump( sd_jurisdiction_crime, open( "SanDiego_crime_by_jurisdiction.p", "wb" ) )

In [6]:
#Checking if the data is now accessible via the pickle
import pandas as pd
sd_crime = pickle.load( open( "SanDiego_crime_by_jurisdiction.p", "rb" ) )
print sd_crime[:3] #Checking the first three rows and formatting
print bool(sd_crime.shape == sd_jurisdiction_crime.shape)

         jurisdiction  homicide rape robbery aggravated_assault  \
year row                                                          
2013 0       Carlsbad       1.0   23      44                153   
     1    Chula Vista       2.0   29     248                316   
     2       Coronado       0.0    2       4                  5   

         residential_burglary non-residential_burglary total_burglary  \
year row                                                                
2013 0                    300                      225            525   
     1                    607                      364            971   
     2                     41                       16             57   

         larceny>$400 larceny>=$400 total_larceny motor_vehicle_theft  arson  \
year row                                                                       
2013 0            596           840         1,436                 124   10.0   
     1          1,280         2,252         3,532         

**More Processing for Statewide Data**

- The PDF data tables summarizing crime in CA from 1952 to 2015 contain crime rates and counts within the same column.
- We need to split these columns apart to get the useful data, and we want to control exactly which columns need splitting.
- `cleantables.split_columns()` will take care of this issue.
    - Note: In development, this function was encapsulated within the batch extraction function, but that proved to be rather unweildy.
    - This function takes in the ouptut from `cleantables.batch_clean_tab_csv()` and puts out the same data frame, but with all of the data values in individual cells.

#### `cleantables.split_columns()`

**Description:** If multiple data values are in a single solumn from the tabula output, splits up and renames these columns.

**Parameters**

**inFrame:** *pandas DataFrame*
- Input DataFrame to have its columns split up

**splitcol:** *list of int, len > 0*
- A non-empty list of indices for columns to be split by word
- The left-most column is index 0, and the index increases by 1 moving to the right.
- Ignores column headings already in the input DataFrame.
- Each column indicated should contain datatype object.
- If the column indicated contains cells with only one word, the one word will go to the rightmost column of the expansion, and the remaining columns contain `None`.

**heads:** *list of str, default=[]*
- A list of new headings, if desired, for the output dataframe
- If left at the default of an empty list, the output headings will be set to numerical integer indices increasing from zero by one moving to the right.
- Will not accept the headings if the output file has a different number of columns than expected. Take a look at the PDF from which the DataFrame is interprted.

**Output:** *pandas DataFrame*
- The same as the original input DataFrame, but with desired columns split up into separate columns by word in object
- Makes sure that the row index remains exactly the same as in the input.